## Lightning in 2 steps


### Good points of Pytorch Lightning

1. Remove Boiler Plates
2. Better Readability
3. Reproducable
4. Less Error Prone
5. Scalable (is this because of multi-gpus?)


### Step 1. Define Lightning Module

Basic Componenet of Lightning Module 

- model
- training_step
- validation_step
- test_step
- predic_step
- configure_optimizers

### Step 2. Fit model with pt.Trainer

Fit model with Trainer is piece of cake

- Load Data
- Define Trainer
- Fit model
- Test model
- Predict using model

### Further readings

- System Examples
- Lightning Modules
- Trainer Modules
- Basic features 
- Other cool features


In [15]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn.functional as F
import pytorch_lightning as pl

class BaseImageClassificationSystem(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.backbone = nn.Sequential(nn.Conv2d(1, 64, 3), nn.AdaptiveAvgPool2d((1,1)))
        self.fc = nn.Linear(64, 10)
        
    def forward(self, x):
        return self.fc(torch.flatten(self.backbone(x), 1))
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.fc(torch.flatten(self.backbone(x), 1))
        loss = F.cross_entropy(y_hat, y)
        self.log('train/loss', loss)
        return loss
    
    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=0.01)

In [16]:
train_dl = DataLoader(datasets.MNIST(root='./', train=True, transform=transforms.ToTensor(), download=True), batch_size=128)
model = BaseImageClassificationSystem()
trainer = pl.Trainer(gpus=1)
trainer.fit(model, train_dl)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name     | Type       | Params
----------------------------------------
0 | backbone | Sequential | 640   
1 | fc       | Linear     | 650   
----------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)


/home/hankyul/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1046: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
